In [1]:
#Libraries used
using NBInclude
using Convex
using LinearAlgebra
using MosekTools

InterruptException: InterruptException:

In [2]:

function getAVID(ID,AV_number,platoon_bool)
    if AV_number == 4
        if platoon_bool > 0
            ID[9] = 1
            ID[10] = 1
            ID[11] = 1
            ID[12] = 1
        else
            ID[3] = 1
            ID[8] = 1
            ID[13] = 1
            ID[18] = 1
        end
    end
            
    if AV_number == 2
        if platoon_bool > 0
            ID[10] = 1
            ID[11] = 1
        else
            ID[5] = 1
            ID[15] = 1
        end
    end
    
    if AV_number == 1
        ID[20] = 1
    end

    return ID
end
    

getAVID (generic function with 1 method)

In [3]:
function ReturnObjectiveValue(AV_ID, N, alpha1, alpha2, alpha3, gammaType)
    # Generate the system model and the optimal objective value
    if(gammaType == 1) 
    #S1
        gamma_s = 0.01
        gamma_v = 0.05
        gamma_u = 0.1
        
    elseif(gammaType == 2) 
    #S2
        gamma_s = 0.03
        gamma_v = 0.15
        gamma_u = 0.1
        
    elseif(gammaType == 3) 
    #S3
        gamma_s = 0.05
        gamma_v = 0.25
        gamma_u = 0.1
         
    elseif(gammaType == 4) 
    #S4
        gamma_s = 0.03
        gamma_v = 0.15
        gamma_u = 1
    
    elseif(gammaType == 5) 
        gamma_s = 1
        gamma_v = 1
        gamma_u = 0
        
    elseif(gammaType == 9999) 
        gamma_s = 0.01
        gamma_v = 0.05
        gamma_u = 1e-6
    end


    AV_number = count(!iszero, AV_ID)
    A1 = [0 -1;alpha1 -alpha2]
    A2 = [0 1; 0 alpha3]
    C1 = [0 -1;0 0]
    C2 = [0 1;0 0]
        
        
    
    A = zeros(2 * N, 2 * N)
    B = zeros(2 * N, AV_number)
    Q = zeros(2 * N, 2 * N)
    
    for i in 1 : N
    Q[2 * i - 1, 2 * i - 1] = gamma_s
    Q[2 * i , 2 * i ] = gamma_v
    end  
    
    R = gamma_u * I(AV_number)
    
    A[1:2,1:2] = A1
    A[1:2,(2 * N - 1):(2 * N)] = A2
    
    for i in 2 : N 
    A[(2 * i - 1): (2 * i) , (2 * i - 1): (2 * i)] = A1
    A[(2 * i - 1): (2 * i) , (2 * i - 3): (2 * i - 2)] = A2
    end
    
    if (alpha2 ^ 2) - (alpha3 ^ 2) - (2 * alpha1) > 0 
    stability_condition_bool = true
    else 
    stability_condition_bool = false
    end
                    
    temp_x = findall(x -> x != 0, (real(eigen(A).values).> 0.001)[1])
    
    if temp_x == 0 
        stable_bool = true
    else 
        stable_bool = false
    end
    
    k = 1
        for i in 1:N
        if AV_ID[i] == 1
            if i == 1
                A[1 : 2, 1 : 2] = C1
                A[1 : 2, (2 * N - 1) : 2 * N] = C2
            else
                A[(2 * i - 1) : (2 * i) , (2 * i - 1) : (2 * i)] = C1
                A[(2 * i - 1) : (2 * i) , (2 * i - 3) : (2 * i - 2)] = C2
            end
            B[2 * i, k] = 1
            k = k + 1
        end
    end
    #print("\n A: \n",A,"\n")
    #print("\n B: \n",B,"\n")
    
    # Call Yalmip to calculate the optimum
    epsilon   = 1e-5
    
    n = length(A[1,:])  # number of states
    m = length(B[1,:]) # number of inputs
    
    #print("\n n: \n",n,"\n")
    #print("\n m: \n",m,"\n")
    
    # assume each vehicle has a deviation
    B1 = I(n)
    B1[begin:2:end, begin:2:end] .= 0
    
    #print("\n B1: \n",B1,"\n")
    
    S = Semidefinite(m + n, m + n)
    #=print("WELLO: \n",S.value)
    print("\n S: \n",Q,"\n")
    print("\n m: \n", R, "\n")
    print("\n n: \n", S.value, "\n")=#
    
    constraint = [(A*S[(m+1):end,(m+1):end] - B*S[begin:m,(m+1):end]) + (A*S[(m+1):end,(m+1):end] 
            - B*S[begin:m,(m+1):end])' + B1*(B1') ⪯ 0,
        S[(m+1):end,(m+1):end] - epsilon*I(n) in :SDP #=,
        S in :SDP=#
    ]
    
    obj = tr(Q*S[m+1:end,m+1:end])+tr(R*S[begin:m,begin:m])
    problem = minimize(obj,constraint)
    solve!(problem,Mosek.Optimizer)
    problem.status
    problem.optval
    #print("\ntesting S: ",S.value,"\n")
    
    Xd = S.value
    Xd = Xd[m+1:end,m+1:end]
    Zd = S.value
    Zd = S[begin:m,m+1:end]
    Yd = S.value
    Yd = S[begin:m,begin:m]
    
    
    
    K = Zd * inv(Xd)
    print("\ntesting obj: ",evaluate(obj),"\n")
    obj = tr(Q*Xd)+tr(R*Yd)
    
    
    #print(K)
    #IOContext(stdout, :limit => false)
    #IOContext(stdout, :displaysize => (100,100))
    #IOContext(stdout, :compact => true)
    
    return obj,stable_bool,stability_condition_bool,K
end


ReturnObjectiveValue (generic function with 1 method)